#### Import required packages

In [43]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#### Import movies dataset
<u>Column Meaning:</u><br>
<i>0 - Movie ID<br>
1 - Name<br>
2 - Genres</i>

In [44]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### Import users dataset
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Gender<br>
2 - Age<br>
3 - Job<br>
4 - Zip Code</i>

In [45]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


#### Import ratings dataset
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [46]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header = None, engine = 'python', encoding = 'latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### Import training set of 80,000 movie ratings and convert to Numpy array
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [47]:
training_set = pd.read_csv('ml-100k/u1.base', header = None, sep = '\t')
training_set = np.array(training_set, dtype=int)
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

#### Import test set of 20,000 movie ratings and convert to Numpy array
<u>Column Meaning:</u><br>
<i>0 - User ID<br>
1 - Movie ID<br>
2 - Ratings (1-5)<br>
3 - Timestamp when movie was rated</i>

In [48]:
test_set = pd.read_csv('ml-100k/u1.test', header = None, sep = '\t')
test_set = np.array(test_set, dtype=int)
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

#### Get the max user ID

In [49]:
num_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
num_users

943

#### Get the max movie ID

In [50]:
num_movies = int(max(max(training_set[:, 1]), max(test_set[:,1])))
num_movies

1682

#### Create a function that will convert the data into an array with users in lines and movies in columns

In [51]:
def convert(data):
    new_data = []
    for user_id in range(1, num_users + 1):
        movie_ids = data[:,1][data[:, 0] == user_id]
        rating_ids = data[:,2][data[:, 0] == user_id]
        ratings = np.zeros(num_movies)
        # Movie IDs start at 1 so we need to subtract 1 to get to base 0 index
        ratings[movie_ids - 1] = rating_ids
        new_data.append(list(ratings))
    return new_data

In [52]:
training_set = convert(training_set)

In [53]:
test_set = convert(test_set)

#### Converting data into Torch tensor

In [54]:
training_set = torch.FloatTensor(training_set)

In [55]:
test_set = torch.FloatTensor(test_set)

#### Creating the AutoEncoder Neural Network Architecture

In [59]:
class SAE(nn.Module): # Stacked AutoEncoder
    def __init__(self, ):
        super(SAE, self).__init__()
        self.full_connection_1 = nn.Linear(num_movies, 20) # 20 nodes in the hidden layer as a suggested start
        self.full_connection_2 = nn.Linear(20, 10) # 10 nodes in the hidden layer as a suggested start
        self.full_connection_3 = nn.Linear(10, 20) # Back to 20
        self.full_connection_4 = nn.Linear(20, num_movies) # Final output equals input
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.full_connection_1(x))
        x = self.activation(self.full_connection_2(x))
        x = self.activation(self.full_connection_3(x))
        x = self.full_connection_4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) # Suggested starting lr and decay

#### Training the Stacked AutoEncoder (SAE)

In [63]:
num_epoch = 200
for epoch in range(1, num_epoch + 1):
    train_loss = 0
    s = 0.
    for user_id in range(num_users):
        input = Variable(training_set[user_id]).unsqueeze(0) # Create a batch of one input vector for network
        target = input.clone()
        if torch.sum(target.data > 0) > 0: # If user has rated at least one movie
            output = sae.forward(input)
            target.require_grad = False
            output[target == 0] = 0 # Do not consider predictions associated with movies not rated by user
            loss = criterion(output, target)
            mean_corrector = num_movies / float(torch.sum(target.data > 0) + 1e-10) # + 1e-10 ensures not nill
            loss.backward()
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print(f'Epoch: {epoch} | Loss: {train_loss/s}')

Epoch: 1 | Loss: 1.754145405461985
Epoch: 2 | Loss: 1.0961239210418803
Epoch: 3 | Loss: 1.0531994729339058
Epoch: 4 | Loss: 1.0384326912746784
Epoch: 5 | Loss: 1.0308131209823361
Epoch: 6 | Loss: 1.026386139312528
Epoch: 7 | Loss: 1.0239515538903534
Epoch: 8 | Loss: 1.022095336474183
Epoch: 9 | Loss: 1.020839908773494
Epoch: 10 | Loss: 1.0196636013724063
Epoch: 11 | Loss: 1.0188403061037414
Epoch: 12 | Loss: 1.0183335186371987
Epoch: 13 | Loss: 1.0177876538700805
Epoch: 14 | Loss: 1.017539947266548
Epoch: 15 | Loss: 1.0172330076105958
Epoch: 16 | Loss: 1.0168428794263185
Epoch: 17 | Loss: 1.0168010939153058
Epoch: 18 | Loss: 1.01643085927196
Epoch: 19 | Loss: 1.0164553985697642
Epoch: 20 | Loss: 1.0162027819578983
Epoch: 21 | Loss: 1.0160199570508042
Epoch: 22 | Loss: 1.0161279160662082
Epoch: 23 | Loss: 1.015790571071349
Epoch: 24 | Loss: 1.0160429333633154
Epoch: 25 | Loss: 1.0156670608048928
Epoch: 26 | Loss: 1.0156500887632944
Epoch: 27 | Loss: 1.0151048898159605
Epoch: 28 | Loss: 

#### Testing the Stacked AutoEncoder (SAE)

In [65]:
test_loss = 0
s = 0.
for user_id in range(num_users):
    input = Variable(training_set[user_id]).unsqueeze(0) # Create a batch of one input vector for network
    target = Variable(test_set[user_id])
    if torch.sum(target.data > 0) > 0: # If user has rated at least one movie
        output = sae.forward(input)
        target.require_grad = False
        output[target.view(1,-1) == 0] = 0 # Do not consider predictions associated with movies not rated by user
        loss = criterion(output, target)
        mean_corrector = num_movies / float(torch.sum(target.data > 0) + 1e-10) # + 1e-10 ensures not nill
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print(f'Test Loss: {test_loss/s}')

Test Loss: 0.9498977348314491
